In [1]:
import numpy as np
import requests
import json
import itertools
import matplotlib.pyplot as plt

# use data on opensteetmap
overpass_url = "http://overpass-api.de/api/interpreter"


In [2]:
# malls
overpass_query = """
[out:json];
area["ISO3166-2"="US-CA"];
(
  node["shop"="mall"](area);
  way["shop"="mall"](area);
  rel["shop"="mall"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data_mall = response.json()

# fitting_room
overpass_query = """
[out:json];
area["ISO3166-2"="US-CA"];
(node["shop"="clothes"](area);
 way["shop"="clothes"](area);
 rel["shop"="clothes"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data_fitting = response.json()
# library 
overpass_query = """
[out:json];
area["ISO3166-2"="US-CA"];
(node["amenity"="hospital"](area);
 way["amenity"="hospital"](area);
 rel["amenity"="hospital"](area);
);
out center;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data_library = response.json()

# basketball
overpass_query = """
[out:json];
area["ISO3166-2"="US-CA"];
(node["amenity"="school"]["name"~"High School"](area);
  way["amenity"="school"]["name"~"High School"](area);
  rel["amenity"="school"]["name"~"High School"](area);
  
  node["amenity"="college"](area);
  way["amenity"="college"](area);
  rel["amenity"="college"](area);

  node["amenity"="university"](area);
  way["amenity"="university"](area);
  rel["amenity"="university"](area);
);
out center;
"""

response = requests.get(overpass_url, params={'data': overpass_query})
data_basketball = response.json()

In [3]:
# Function to calculate Ripley's K
def ripley_k(data, r):
    n = len(data)
    k = np.zeros(len(r))
    for i in range(n):
        for j, radius in enumerate(r):
            in_circle = np.linalg.norm(data[i] - data, axis=1) <= radius
            k[j] += np.sum(in_circle)
    return k / n

# Radius values
r = np.linspace(0, 35, 100)

In [4]:
import numpy as np

# 函数来从JSON响应中提取坐标数据
def extract_coordinates(json_data):
    elements = json_data['elements']
    coordinates = np.array([[element['lat'], element['lon']] for element in elements])
    return coordinates

# 将数据保存为.npy文件的函数
def save_data(data, filename):
    np.save(filename, data)

fitting_coordinates = extract_coordinates(data_fitting)
save_data(fitting_coordinates, "data_fitting.npy")

mall_coordinates = extract_coordinates(data_mall)
save_data(mall_coordinates, "data_mall.npy")

basketball_coordinates = extract_coordinates(data_basketball)
save_data(basketball_coordinates, "data_basketball.npy")

library_coordinates = extract_coordinates(data_library)
save_data(library_coordinates, "data_library.npy")

data_basketball_1 = np.load("data_basketball.npy")
data_library_1 = np.load("data_library.npy")
data_mall_1 = np.load("data_mall.npy")
data_fitting_1 = np.load("data_fitting.npy")

# 合并数据集
data_combined = np.concatenate([data_basketball_1, data_library_1, data_mall_1, data_fitting_1], axis=0)

# 保存组合的数据备用
save_data(data_combined, "data_combined.npy")


In [5]:
# Display Ripley’s K Curve
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

In [7]:
# Calculate Ripley's K for different datasets
from scipy.stats import poisson


k_basketball = ripley_k(data_basketball_1, r)
k_library = ripley_k(data_library_1, r)
k_mall = ripley_k(data_mall_1, r)
k_fitting = ripley_k(data_fitting_1, r)

# Calculate K function for CSR process
k_csr = np.pi * r**2

# Calculate high and low boundaries for 95% confidence interval
upper_bound = poisson.ppf(0.975, mu=k_csr)
lower_bound = poisson.ppf(0.025, mu=k_csr)

In [8]:
min_lat, max_lat = 32.5, 42.0
min_lon, max_lon = -124.5, -114.0

num_lat_squares = 30
num_lon_squares = 60

lat_step = (max_lat - min_lat) / num_lat_squares
lon_step = (max_lon - min_lon) / num_lon_squares

k_matrix = np.zeros((num_lat_squares, num_lon_squares, 4))

In [9]:
def get_square(lat, lon, min_lat, min_lon, lat_step, lon_step):
    lat_idx = int((lat - min_lat) / lat_step)
    lon_idx = int((lon - min_lon) / lon_step)
    return lat_idx, lon_idx

def assign_k_values(data, k_values, k_index, k_matrix):
    for (lat, lon), k_value in zip(data, k_values):
        lat_idx, lon_idx = get_square(lat, lon, min_lat, min_lon, lat_step, lon_step)
        k_matrix[lat_idx, lon_idx, k_index] += k_value

assign_k_values(data_mall_1, k_mall, 0, k_matrix)
assign_k_values(data_library_1, k_library, 1, k_matrix)
assign_k_values(data_fitting_1, k_fitting, 2, k_matrix)
assign_k_values(data_basketball_1, k_basketball, 3, k_matrix)

In [10]:
def determine_threshold(k_matrix):
    k_values = k_matrix.flatten()
    mean_k = np.mean(k_values)
    std_k = np.std(k_values)
    threshold = mean_k + std_k
    return threshold

threshold = determine_threshold(k_matrix)

In [11]:
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def is_valid_combination(comb, k_matrix, min_lat, min_lon, lat_step, lon_step, threshold):
    seen_blocks = set()
    for place in comb:
        block_index = get_square(place[0], place[1], min_lat, min_lon, lat_step, lon_step)
        if block_index in seen_blocks:
            return False
        if np.any(k_matrix[block_index[0], block_index[1], :] < threshold):
            return False
        seen_blocks.add(block_index)
    return True

def find_shortest_path(malls, libraries, fitting_studios, basketball_courts, k_matrix, min_lat, min_lon, lat_step, lon_step):
    combinations = itertools.product(malls, libraries, fitting_studios, basketball_courts)
    min_distance = float('inf')
    best_path = None
    for comb in combinations:
        if is_valid_combination(comb, k_matrix, min_lat, min_lon, lat_step, lon_step, threshold):
            distance = 0
            for i in range(len(comb) - 1):
                current_point = np.array(comb[i])
                next_point = np.array(comb[i+1])
                current_distance = calculate_distance(current_point, next_point)
                distance += current_distance
            if distance < min_distance:
                min_distance = distance
                best_path = comb
                print(f"New best path: {best_path} with distance: {min_distance}")
    return best_path, min_distance

In [12]:
best_path, min_distance = find_shortest_path(data_mall_1, data_library_1, data_fitting_1, data_basketball_1, k_matrix, min_lat, min_lon, lat_step, lon_step)
print("Best Path:", best_path)
print("Minimum Distance:", min_distance)

KeyboardInterrupt: 